In [1]:
%%time
import platform
import boto3
import pandas as pd

version = int(platform.python_version_tuple()[0])
if version < 3:
    from StringIO import StringIO
else:
    from io import StringIO

BUCKET_NAME_GLUE = 'attemp-glue'
BUCKET_NAME_CSVS = 'charbeat-trafic'
path_work = 'gestion.pe/2018/12/'

s3 = boto3.resource('s3')
bucket_csv = s3.Bucket(BUCKET_NAME_CSVS)

list_df = []
col_analysis = 'page_avg_time'
df_up = pd.DataFrame(columns=['dia', col_analysis])

for obj in bucket_csv.objects.filter(Prefix=path_work):
    key = obj.key
    print(key)
    url_up = 'https://s3.amazonaws.com/{}/{}'.format(BUCKET_NAME_CSVS, key)
    df_s3 = pd.read_csv(url_up, parse_dates=['dia'])[['dia', col_analysis]]
    df_s3['mes'] = df_s3['dia'].dt.month 
    
    csv_buffer = StringIO()
    df_s3.to_csv(csv_buffer, index=False)
    s3.Object(BUCKET_NAME_GLUE, key).put(Body=csv_buffer.getvalue(), ACL='public-read')
    # print(help(s3.Object(BUCKET_NAME_GLUE, key)))
    """
    copy_source = {
        'Bucket': BUCKET_NAME_CSVS,
        'Key': key
    }
    s3.meta.client.copy(copy_source, BUCKET_NAME_GLUE, key)
    """

gestion.pe/2018/12/2018-12-18.csv
gestion.pe/2018/12/2018-12-19.csv
gestion.pe/2018/12/2018-12-20.csv
gestion.pe/2018/12/2018-12-21.csv
gestion.pe/2018/12/2018-12-22.csv
gestion.pe/2018/12/2018-12-23.csv
gestion.pe/2018/12/2018-12-24.csv
gestion.pe/2018/12/2018-12-25.csv
gestion.pe/2018/12/2018-12-26.csv
gestion.pe/2018/12/2018-12-27.csv
gestion.pe/2018/12/2018-12-28.csv
gestion.pe/2018/12/2018-12-29.csv
gestion.pe/2018/12/2018-12-30.csv
gestion.pe/2018/12/2018-12-31.csv
CPU times: user 4.4 s, sys: 669 ms, total: 5.07 s
Wall time: 1min 12s


In [2]:
baz = {'a': 1, 'b': 2}
foo, bar = baz
print(foo, bar)

a b


In [3]:
baz = [1, 2]
foo, bar = baz
print(foo, bar)

1 2


In [4]:
baz = 3, 4
foo, bar = baz
print(foo, bar)

3 4


In [91]:
%%time
## VERSION WITH DASK
import dask.dataframe as dd
import pandas as pd

bucket_name = "aatempjson2"
hour = 19
path_json_spark = 's3://{}/{}/{}/{}/{}/*'.format(
    bucket_name, '2019', '04', '08', hour
)
df = dd.read_csv(
    path_json_spark, sep='@@@', engine='python', header=None, squeeze=False
).compute()

CPU times: user 211 ms, sys: 24.8 ms, total: 236 ms
Wall time: 2.67 s


In [92]:
def get_memory_usage(data, deep=True):
    return '{} MB'.format(data.memory_usage(deep=deep).sum() / 1024 ** 2)

get_memory_usage(df)

'0.4536914825439453 MB'

In [93]:
df.shape

(2, 1)

In [94]:
unique_col = df.columns[0]
df.head()

,0
0,"{""project"":""ojo_dev"",""event"":""receive"",""date"":..."
0,"{""project"":""ojo_dev"",""event"":""receive"",""date"":..."


In [95]:
df = pd.DataFrame(data=''.join(list(df[unique_col])).replace('}{', '} {').split(' '))
df.head()

,0
0,"{""project"":""ojo_dev"",""event"":""receive"",""date"":..."
1,"{""project"":""ojo_dev"",""event"":""receive"",""date"":..."
2,"{""project"":""ojo_dev"",""event"":""receive"",""date"":..."
3,"{""project"":""ojo_dev"",""event"":""receive"",""date"":..."
4,"{""project"":""ojo_dev"",""event"":""ctr"",""date"":""201..."


In [96]:
import json
unique_col = df.columns[0]
df[unique_col] = df[unique_col].apply(json.loads)

In [97]:
df.head()

,0
0,"{'project': 'ojo_dev', 'event': 'receive', 'da..."
1,"{'project': 'ojo_dev', 'event': 'receive', 'da..."
2,"{'project': 'ojo_dev', 'event': 'receive', 'da..."
3,"{'project': 'ojo_dev', 'event': 'receive', 'da..."
4,"{'project': 'ojo_dev', 'event': 'ctr', 'date':..."


In [98]:
get_memory_usage(df)

'1.2390899658203125 MB'

In [99]:
for col in df[unique_col][0].keys():
    df[col] = df[unique_col].apply(lambda _: _.get(col, ''))

del df[unique_col]

In [100]:
get_memory_usage(df), df.shape

('1.736907958984375 MB', (5800, 5))

In [101]:
df['hour'] = hour
df['date'] = pd.to_datetime(df['date'], format="%Y%m%d")
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df = df[['project', 'campain', 'url', 'event', 'year', 'month', 'day', 'hour']]
df.head()

,project,campain,url,event,year,month,day,hour
0,ojo_dev,xxxxx,,receive,2019,4,5,19
1,ojo_dev,xxxxx,,receive,2019,4,5,19
2,ojo_dev,xxxxx,,receive,2019,4,5,19
3,ojo_dev,xxxxx,,receive,2019,4,5,19
4,ojo_dev,xxxxx,,ctr,2019,4,5,19


In [102]:
get_memory_usage(df), df.shape

('1.5543746948242188 MB', (5800, 8))

In [103]:
df = [tuple(_) for _ in df.values]
df

[('ojo_dev', 'xxxxx', '', 'receive', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'receive', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'receive', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'receive', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'receive', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'receive', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'receive', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'receive', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'receive', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', '

In [104]:
from datetime import datetime
datetime.utcnow().hour

21